In [45]:
import keras
from keras import layers
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Activation, AveragePooling2D
from keras import regularizers
from keras.optimizers import Adam
import pandas as pd

In [46]:
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn import preprocessing
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
kernel = 1.0 * RBF(1.0)
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
from sklearn import preprocessing


In [47]:
# load the dataset
import pandas as pd
from scipy.io import arff
data = arff.loadarff('/content/drive/MyDrive/final_paper/MLL.arff') # selcet labeled data according to application
df = pd.DataFrame(data[0])
df.shape

(72, 12583)

In [48]:
from sklearn import preprocessing
print( df.iloc[:,-1].unique()) # to check number of classes in the dataset
# label_encoder object knows how to understand word labels.
label_encoder = preprocessing.LabelEncoder()
# Encode labels in column 'species'.
df.iloc[:,-1]= label_encoder.fit_transform(df.iloc[:,-1])
df.iloc[:,-1].unique()

[b'ALL' b'MLL' b'AML']


array([0, 2, 1], dtype=object)

In [49]:
# separate dataset according to classes
x1 = df[df.iloc[:,-1] == 0]
x2 = df[df.iloc[:,-1] == 1]
x3 = df[df.iloc[:,-1] == 2]
print(x1.shape,x2.shape,x3.shape) # to check number of samples in each class

(24, 12583) (28, 12583) (20, 12583)


In [50]:
#separate the target classes
Y1 = x1.iloc[:,-1]
Y2 = x2.iloc[:,-1]
Y3 = x3.iloc[:,-1]
# remove the class label
del x1[x1.columns[-1]]
del x2[x2.columns[-1]]
del x3[x3.columns[-1]]
print(x1.shape,x2.shape,x3.shape) # to check number of samples in each class without class labels

(24, 12582) (28, 12582) (20, 12582)


In [51]:
#Select samples for augmentation
n=10
df1=x1.sample(n =10)
df2=x2.sample(n = 10)
df3=x3.sample(n = 10)


In [52]:
import numpy as np
mu, sigma = 0, 0.1
# creating a noise with the same dimension as the dataset
noise1 = np.random.normal(mu, sigma, [10,x1.shape[1]])
noise2 = np.random.normal(mu, sigma, [10,x1.shape[1]])
noise3 = np.random.normal(mu, sigma, [10,x1.shape[1]])
print(noise1.shape,noise2.shape,noise3.shape) # check the shape of the noise

(10, 12582) (10, 12582) (10, 12582)


In [54]:
# add noise to selected samples
noise_factor = 0.1
DF1 =  df1+ (noise_factor*noise1)
DF2=   df2+ (noise_factor*noise2)
DF3=   df3+ (noise_factor*noise3)


In [55]:
# add noisy sample to original data
verti_df1= pd.concat([x1, DF1], axis=0)
verti_df2= pd.concat([x2, DF2], axis=0)
verti_df3= pd.concat([x3, DF3], axis=0)
print(verti_df1.shape,verti_df2.shape,verti_df3.shape)# check the shape of the augmented data

(34, 12582) (38, 12582) (30, 12582)


In [56]:
#merge all samples to new augmented data
new_data=pd.concat([verti_df1,verti_df2,verti_df3],axis=0)
new_data.shape

(102, 12582)

In [57]:
#create equal class labels
y1= Y1.sample(n = 10)
y2= Y2.sample(n = 10)
y3= Y3.sample(n = 10)

new_y1= pd.concat([Y1,y1],axis=0)
new_y2=pd.concat([Y2,y2],axis=0)
new_y3=pd.concat([Y3,y3],axis=0)
new_Class= pd.concat([new_y1,new_y2,new_y3],axis=0)
new_Class.shape

(102,)

In [58]:
#merge class label to dataset to create augmented dataset(data with label)
from sklearn.utils import shuffle
NEW_dataset=new_data.reset_index(drop=True).merge(new_Class.reset_index(drop=True), left_index=True, right_index=True)
NEW_dataset= (shuffle(NEW_dataset)).reset_index(drop=True)

In [59]:
print(NEW_dataset.shape)
dataset= NEW_dataset.values
X = dataset[:,:-1]
Y = dataset[:,-1]
print(X.shape,Y.shape)
input_dim=X.shape[1]
print(input_dim)

(102, 12583)
(102, 12582) (102,)
12582


In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20)
scaler = MinMaxScaler(feature_range=(0, 1))
X_scl_train = scaler.fit_transform(X_train)
X_scl_test = scaler.fit_transform(X_test)
X_scl = scaler.fit_transform(X)

In [60]:
input_data = keras.Input(shape=(input_dim,))
encoded = layers.Dense(1024, activation='relu')(input_data)
output = layers.Dense(input_dim, activation='sigmoid')(encoded)
en_m=keras.Model(input_data,encoded)
#encoder_m.summary()
auto_m = keras.Model(input_data, output)
auto_m.summary()

Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 12582)]           0         
                                                                 
 dense_9 (Dense)             (None, 1024)              12884992  
                                                                 
 dense_10 (Dense)            (None, 12582)             12896550  
                                                                 
Total params: 25781542 (98.35 MB)
Trainable params: 25781542 (98.35 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [61]:
auto_m.compile(optimizer='adam', loss='binary_crossentropy')
auto_m.fit(X_scl_train, X_scl_train,
                epochs=100,
                batch_size=30,
                shuffle=True,
                validation_data=(X_scl_test, X_scl_test))

Epoch 1/100
3/3 [==============================] - 3s 842ms/step - loss: 0.6826 - val_loss: 0.6820
Epoch 2/100
3/3 [==============================] - 2s 819ms/step - loss: 0.6349 - val_loss: 0.6698
Epoch 3/100
3/3 [==============================] - 2s 813ms/step - loss: 0.6228 - val_loss: 0.6707
Epoch 4/100
3/3 [==============================] - 2s 752ms/step - loss: 0.6169 - val_loss: 0.6594
Epoch 5/100
3/3 [==============================] - 2s 541ms/step - loss: 0.6084 - val_loss: 0.6474
Epoch 6/100
3/3 [==============================] - 2s 761ms/step - loss: 0.6027 - val_loss: 0.6393
Epoch 7/100
3/3 [==============================] - 2s 691ms/step - loss: 0.6001 - val_loss: 0.6386
Epoch 8/100
3/3 [==============================] - 2s 583ms/step - loss: 0.5965 - val_loss: 0.6381
Epoch 9/100
3/3 [==============================] - 2s 778ms/step - loss: 0.5931 - val_loss: 0.6406
Epoch 10/100
3/3 [==============================] - 2s 775ms/step - loss: 0.5907 - val_loss: 0.6394
Epoch 11/

In [62]:
#adding two new layers autoencoder 2
en2=(Dense(512,activation='relu'))(auto_m.layers[-2].output)
de2=(Dense(1024,activation='relu'))(en2)
ou=(Dense(input_dim,activation='sigmoid'))(de2)
#creating new model
ae_model=keras.Model(inputs=auto_m.input, outputs=[ou])
encoder_model=keras.Model(inputs=auto_m.input, outputs=[en2])
encoder_model.summary()
ae_model.summary()

Model: "model_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 12582)]           0         
                                                                 
 dense_9 (Dense)             (None, 1024)              12884992  
                                                                 
 dense_11 (Dense)            (None, 512)               524800    
                                                                 
Total params: 13409792 (51.15 MB)
Trainable params: 13409792 (51.15 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Model: "model_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 12582)]           0         
                                                                 


In [63]:
#train new model
ae_model.compile(optimizer='adam', loss='binary_crossentropy')
ae_model.fit(X_scl_train, X_scl_train,
                epochs=100,
                batch_size=30,
                shuffle=True,
                validation_data=(X_scl_test, X_scl_test))

Epoch 1/100
3/3 [==============================] - 3s 575ms/step - loss: 0.6829 - val_loss: 0.6756
Epoch 2/100
3/3 [==============================] - 1s 509ms/step - loss: 0.6338 - val_loss: 0.6583
Epoch 3/100
3/3 [==============================] - 1s 505ms/step - loss: 0.6190 - val_loss: 0.6617
Epoch 4/100
3/3 [==============================] - 1s 508ms/step - loss: 0.6070 - val_loss: 0.6459
Epoch 5/100
3/3 [==============================] - 2s 524ms/step - loss: 0.5945 - val_loss: 0.6394
Epoch 6/100
3/3 [==============================] - 2s 512ms/step - loss: 0.5865 - val_loss: 0.6346
Epoch 7/100
3/3 [==============================] - 2s 661ms/step - loss: 0.5804 - val_loss: 0.6311
Epoch 8/100
3/3 [==============================] - 2s 791ms/step - loss: 0.5746 - val_loss: 0.6260
Epoch 9/100
3/3 [==============================] - 2s 759ms/step - loss: 0.5698 - val_loss: 0.6243
Epoch 10/100
3/3 [==============================] - 2s 503ms/step - loss: 0.5653 - val_loss: 0.6203
Epoch 11/

In [64]:
#adding two new layers
en3=(Dense(100,activation='relu'))(ae_model.layers[-3].output)
de3=(Dense(512,activation='relu'))(en3)
de3=(Dense(1024,activation='relu'))(de3)
ou3=(Dense(input_dim,activation='sigmoid'))(de3)

In [65]:
#creating final stacked model
ae_model_new=keras.Model(inputs=auto_m.input, outputs=[ou3])
SAE=keras.Model(inputs=auto_m.input, outputs=[ae_model_new.layers[-4].output])
ae_model_new.summary()

Model: "model_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 12582)]           0         
                                                                 
 dense_9 (Dense)             (None, 1024)              12884992  
                                                                 
 dense_11 (Dense)            (None, 512)               524800    
                                                                 
 dense_14 (Dense)            (None, 100)               51300     
                                                                 
 dense_15 (Dense)            (None, 512)               51712     
                                                                 
 dense_16 (Dense)            (None, 1024)              525312    
                                                                 
 dense_17 (Dense)            (None, 12582)             128

In [66]:
#train final stacked model
ae_model_new.compile(optimizer='adam', loss='binary_crossentropy')
ae_model_new.fit(X_scl_train, X_scl_train,
                epochs=100,
                batch_size=30,
                shuffle=True,
                validation_data=(X_scl_test, X_scl_test))

Epoch 1/100
3/3 [==============================] - 3s 594ms/step - loss: 0.6851 - val_loss: 0.6680
Epoch 2/100
3/3 [==============================] - 2s 515ms/step - loss: 0.6484 - val_loss: 0.6625
Epoch 3/100
3/3 [==============================] - 1s 499ms/step - loss: 0.6283 - val_loss: 0.6653
Epoch 4/100
3/3 [==============================] - 3s 871ms/step - loss: 0.6214 - val_loss: 0.6638
Epoch 5/100
3/3 [==============================] - 2s 814ms/step - loss: 0.6077 - val_loss: 0.6468
Epoch 6/100
3/3 [==============================] - 2s 501ms/step - loss: 0.5995 - val_loss: 0.6413
Epoch 7/100
3/3 [==============================] - 1s 493ms/step - loss: 0.5935 - val_loss: 0.6380
Epoch 8/100
3/3 [==============================] - 1s 506ms/step - loss: 0.5878 - val_loss: 0.6334
Epoch 9/100
3/3 [==============================] - 1s 498ms/step - loss: 0.5821 - val_loss: 0.6305
Epoch 10/100
3/3 [==============================] - 2s 524ms/step - loss: 0.5771 - val_loss: 0.6279
Epoch 11/

In [67]:
X_new = SAE.predict(X_scl)
print(X_new.shape)

4/4 [==============================] - 0s 28ms/step
(102, 100)


In [ ]:
cv = KFold(n_splits=2, random_state=1, shuffle=True)
rf_model = RandomForestClassifier(n_estimators=500, random_state=0)
LR = LogisticRegression(random_state=0)
lda_clf = LinearDiscriminantAnalysis()
qda_clf = QuadraticDiscriminantAnalysis()
gpc = GaussianProcessClassifier(kernel=kernel,random_state=0)
svm_model = svm.SVC(random_state=1,kernel='rbf', probability=True)
gnb = GaussianNB()
dt_model = DecisionTreeClassifier(random_state=0)
adb = AdaBoostClassifier(n_estimators=100, random_state=0)
modls=[rf_model,LR,lda_clf,qda_clf,gpc,svm_model,gnb,dt_model,adb]
names=["RF", "LR", "LDA", "QDA", "GPC", "SVM", "NB", "DT", "AB"]

In [ ]:
#X,Y=load_dataset(df)
X=X_new
size=X.shape[1]
print(X.shape)
#ws=create_ws(WB,'sae')
#ws.write(0,0,  (size) )
print('    Accuracy: recall: precision : F1 score: kappa: AUC  ' )
for mo in range(9):
    print(names[mo])
    row=mo
    col=1
    scores = cross_val_score(modls[mo], X, Y, scoring='accuracy', cv=cv, n_jobs=-1)
    p_scores = cross_val_score(modls[mo], X, Y, scoring='precision_macro', cv=cv, n_jobs=-1)
    r_scores = cross_val_score(modls[mo], X, Y, scoring='recall_macro', cv=cv, n_jobs=-1)
    f_scores = cross_val_score(modls[mo], X, Y, scoring='f1_macro', cv=cv, n_jobs=-1)
    acc=(mean(scores)*100)
    re= (mean(r_scores)*100)
    pre=(mean(p_scores)*100)
    f1=(mean(f_scores)*100)
    print('    %.2f   %.2f   %.2f   %.2f' %(acc, re, pre, f1))


In [68]:
#find the hightest weighted genes
weight = SAE.get_weights()
we_ts1=np.asarray(weight[0])
we_ts2=np.asarray(weight[2])
we_ts3=np.asarray(weight[4])
aa=np.dot(we_ts1,we_ts2)
result=np.dot(aa,we_ts3)
result.shape

(12582, 100)

In [69]:
total_gene=result.shape[0]
total_node=result.shape[1]
gene_weight_list=[]
for t_g in range(total_gene):
    weight_sum=0

    for node in range(total_node):
        weight_sum=weight_sum+result[t_g][node]

    #print(t_g)
    #print(weight_sum)
    gene_weight_list.append((t_g,weight_sum))
print(t_g)

12581


In [70]:
gene_list=sorted(gene_weight_list,key=lambda gene_weight_list: gene_weight_list[1] ,reverse=True)
top_gene=gene_list[:50]
gen_data = [lis[0] for lis in top_gene]
gene_data=np.asarray(gen_data)
col_name=[]
for col in NEW_dataset.columns:
    col_name.append(col)
gene_name=[]
for ge_name in gene_data:
    gene_name.append(col_name[ge_name])

In [71]:
#hightest weighted genes
gene_name

['38355_at',
 '39296_at',
 '34526_s_at',
 '315_at',
 '38319_at',
 '37078_at',
 '32438_at',
 '40396_at',
 '33238_at',
 '32649_at',
 '1633_g_at',
 '41273_at',
 '1021_at',
 '34831_at',
 '35832_at',
 '36630_at',
 '41164_at',
 '39771_at',
 '33405_at',
 '40203_at',
 '34224_at',
 '1961_f_at',
 '31593_at',
 '854_at',
 '33267_at',
 '316_g_at',
 '39643_at',
 'AFFX-M27830_5_at',
 '40324_r_at',
 '39042_at',
 '33291_at',
 '347_s_at',
 '33130_at',
 '1972_s_at',
 '41544_at',
 '35936_g_at',
 '1106_s_at',
 '586_s_at',
 '31754_at',
 '33518_f_at',
 '39829_at',
 '35862_at',
 '31885_at',
 '40531_at',
 '34037_at',
 '31957_r_at',
 '33616_at',
 '41042_r_at',
 '31382_f_at',
 '31665_s_at']